In [1]:
import os
import re
import ftfy
import nltk
import smtplib
import numpy as np
import pandas as pd
import pickle as pkl
import lib_file
from pathlib import Path
from nltk import PorterStemmer
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences

Welcome to Python
3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]


Using TensorFlow backend.


In [2]:
input_dir = 'input'
model_dir = 'model'
model_str_file = 'model_structure.json'
model_weights_file = 'model_weights.h5'
# embedding_file = 'google_news_vectors_negative_300.bin'
cword_file = 'cword_dict.pkl'
tokenizer_file = 'tokens.pkl'

In [3]:
class_names = [
    'Not-Depressed',
    'Depressed'
]
print(class_names)

['Not-Depressed', 'Depressed']


In [4]:
#word2vec = KeyedVectors.load_word2vec_format(os.path.join(input_dir, embedding_file), binary=True)
cList = pkl.load(open(os.path.join(input_dir,cword_file),'rb'))

In [5]:
trained_tokenizer = pkl.load(open(os.path.join(model_dir,tokenizer_file),'rb'))

In [6]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [7]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [8]:
from nltk.corpus import stopwords

In [9]:
#nltk.download()

In [10]:
def clean_tweets(tweets):
    cleaned_tweets = []
    for tweet in tweets:
        tweet = str(tweet)
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())
            tweet = ftfy.fix_text(tweet)
            tweet = expandContractions(tweet)
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())
            stop_words = set(stopwords.words('english'))
            word_tokens = nltk.word_tokenize(tweet) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            tweet = ' '.join(filtered_sentence)
            tweet = PorterStemmer().stem(tweet)
            cleaned_tweets.append(tweet)
    return cleaned_tweets

In [11]:
def load_trained_model(model_str_path, model_wt_path):
    f = Path(model_str_path)
    model_structure = f.read_text()
    trained_model = model_from_json(model_structure)
    trained_model.load_weights(model_wt_path)
    return trained_model

In [12]:
str_path = os.path.join(model_dir,model_str_file)
wt_path = os.path.join(model_dir,model_weights_file)

In [14]:
trained_model = load_trained_model(str_path, wt_path)

In [24]:
input_string = input("Enter text")

Enter textam in a good mood


In [25]:
test_data  = input_string.split(",")

In [26]:
print(test_data)

['am in a good mood']


In [27]:
cleaned_text = clean_tweets(test_data)
print(cleaned_text)

['good mood']


In [28]:
sequences_text_token = trained_tokenizer.texts_to_sequences(cleaned_text)
print(sequences_text_token)

[[8, 309]]


In [29]:
data = pad_sequences(sequences_text_token, maxlen=140)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (1, 140)


In [30]:
data

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   8, 309]])

In [31]:
from IPython.core.display import HTML
def predict_result(data, model):
    result = model.predict(data)
    print(result)
    Y_pred = np.round(result.flatten())
    print(Y_pred)
    print(' -^- '*20)
    for i in range(len(Y_pred)):
        print("Text : ",cleaned_text[i])
        print("Result : ",class_names[int(Y_pred[i])])
       
        print('-'*40)
    print(' -^- '*20)

In [32]:
Y_pred=predict_result(data=data, model= trained_model)

[[0.00515186]]
[0.]
 -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^- 
Text :  good mood
Result :  Not-Depressed
----------------------------------------
 -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^-  -^- 


# Thank You...